<a href="https://colab.research.google.com/github/Engr-Muhammad-Anees/Dubbing-Podcast-ML/blob/main/dubbing_podcast_by_using_assembly_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**package install:**

In [ ]:
!pip install moviepy assemblyai elevenlabs pydub

**import modules/libraries:**

In [ ]:
import os
import io
import assemblyai as aai
from moviepy.editor import VideoFileClip, AudioFileClip
from elevenlabs import ElevenLabs
from pydub import AudioSegment

**API KEYS:**

In [ ]:
ASSEMBLYAI_API_KEY = "78ea673ef72d4588ad6c61e08fdb5d37"
ELEVENLABS_API_KEY = "sk_3f810b2dda995bc3a77580f2b8e5e61f9ac35a3b1395c417"

**INPUT/OUTPUT VIDEO PATH ASSIGN:**

In [ ]:
original_video_path = "/content/drive/MyDrive/video for dubbing podcast/input_video_2.mp4"
output_audio_path   = "/content/drive/MyDrive/video for dubbing podcast/final_dubbed_audio2.wav"
output_video_path   = "/content/drive/MyDrive/video for dubbing podcast/final_dubbed_video2.mp4"

**SELECT VOICE ID:**

In [ ]:
speaker_voices = {
    "A": "pNInz6obpgDQGcFmaJgB",
    "B": "TxGEqnHWrfWFTfGW9XjX",
    "SPEAKER_0": "pNInz6obpgDQGcFmaJgB",
    "SPEAKER_1": "TxGEqnHWrfWFTfGW9XjX"
}

**Extract audio from video**

In [ ]:
video = VideoFileClip(original_video_path)
audio_path = "extracted_audio.wav"
video.audio.write_audiofile(audio_path, verbose=False, logger=None)
print(f" Video duration: {video.duration:.2f}s")

 Video duration: 36.08s


**Transcribe + diarize using AssemblyAI:**

In [ ]:
transcriber = aai.Transcriber()
config = aai.TranscriptionConfig(speaker_labels=True, speakers_expected=2)
transcript = transcriber.transcribe(audio_path, config)

segments = transcript.utterances
print(f"Total segment detect: {len(segments)}")


Total segment detect: 12


**Prepare ElevenLabs client:**

In [ ]:
client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

**summary diarized segments:**

**Generate dubbed audio per segment:**

In [ ]:
!pip install moviepy elevenlabs pydub

In [ ]:
import time

In [ ]:
total_duration_ms = int(video.duration * 1000)
final_dub_audio = AudioSegment.silent(duration=total_duration_ms)

for seg in segments:
    speaker = seg.speaker
    text = seg.text.strip()
    start_ms = int(seg.start)
    end_ms   = int(seg.end)
    duration_ms = max(200, end_ms - start_ms)
    voice_id = speaker_voices.get(speaker, "pNInz6obpgDQGcFmaJgB")

    print(f"\n Generating {speaker} ({start_ms}-{end_ms} ms | {duration_ms/1000:.2f}s): '{text}'")

    seg_audio = None
    attempt = 0

    while attempt < 2 and seg_audio is None:
        try:
            audio_stream = client.text_to_speech.convert(
                voice_id=voice_id,
                model_id="eleven_multilingual_v2",
                text=text
            )

            audio_bytes = b"".join(audio_stream)
            if not audio_bytes:
                raise ValueError("Empty stream received from ElevenLabs")

            seg_audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")

        except Exception as e:
            attempt += 1
            print(f" Error (attempt {attempt}) for {speaker}: {e}")
            time.sleep(2)

    if seg_audio is None:
        print(f" ElevenLabs failed for '{text}', inserting {duration_ms}ms silence.")
        seg_audio = AudioSegment.silent(duration=duration_ms)

    final_dub_audio = final_dub_audio.overlay(seg_audio, position=start_ms)


 Generating A (240-1440 ms | 1.20s): 'Hello. How are you?'

 Generating B (2000-4400 ms | 2.40s): 'I'm fine, thank you. How are you?'

 Generating A (4880-8000 ms | 3.12s): 'I'm good, thanks. What's your name?'

 Generating B (8640-10720 ms | 2.08s): 'My name is Raj. What's yours?'

 Generating A (11200-14160 ms | 2.96s): 'My name is Saloni. How old are you?'

 Generating B (14640-17600 ms | 2.96s): 'I am 25 years old. How about you?'

 Generating A (18000-21200 ms | 3.20s): 'I am 30 years old. Where are you from?'

 Generating B (21760-23840 ms | 2.08s): 'I am from Delhi. Where are you from?'

 Generating A (24320-27680 ms | 3.36s): 'I am from Chandigarh. What do you do for a living?'

 Generating B (28580-30500 ms | 1.92s): 'I am a teacher. What about you?'

 Generating A (30900-32180 ms | 1.28s): 'I am a student.'

 Generating B (32660-35380 ms | 2.72s): 'Essay or video? Scalia. English Academico. Subscribe Curry.'


In [ ]:
final_dub_audio

In [ ]:
dubbed_ms = len(final_dub_audio)
video_ms = int(video.duration * 1000)

if dubbed_ms < video_ms:
    final_dub_audio += AudioSegment.silent(duration=video_ms - dubbed_ms)
elif dubbed_ms > video_ms:
    final_dub_audio = final_dub_audio[:video_ms]

In [ ]:
final_dub_audio.export(output_audio_path, format="wav")
print(f"\n Dubbed audio saved at: {output_audio_path}")


 Dubbed audio saved at: /content/drive/MyDrive/video for dubbing podcast/final_dubbed_audio2.wav


**Merge dubbed audio back into video**

In [ ]:
dub_audio = AudioFileClip(output_audio_path)
final_video = video.set_audio(dub_audio)
final_video.write_videofile(output_video_path, codec="libx264", audio_codec="aac")

print(f"\n Final dubbed video saved at: {output_video_path}")

Moviepy - Building video /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video2.mp4.
MoviePy - Writing audio in final_dubbed_video2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video2.mp4



t:   0%|          | 0/902 [00:00<?, ?it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/video for dubbing podcast/input_video_2.mp4, 691200 bytes wanted but 0 bytes read,at frame 44/903, at time 1.76/36.08 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:   5%|▌         | 46/902 [00:00<00:02, 399.94it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/drive/MyDrive/video for dubbing podcast/input_video_2.mp4, 691200 bytes wanted but 0 bytes read,at frame 45/903, at time 1.80/36.08 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in fil

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video2.mp4

 Final dubbed video saved at: /content/drive/MyDrive/video for dubbing podcast/final_dubbed_video2.mp4
